# 2. Build a Recommendation System for Purchase Data

The scope of this notebook is 

- Code the Scoring Function
- Unit Test the Score
- Build the Dash Applimcation



## Settings

In [34]:
#Model Tracking
import mlflow
from mlflow.tracking import MlflowClient

#Utils
from pprint import pprint

In [66]:
#Enviroment variables

outmodels = '../models/'

## Download the model from the registry

In [71]:
client = MlflowClient()
for regmodel in client.list_registered_models():
    regmodel_info = dict(regmodel)

pprint(regmodel_info, indent=3)

# run_id = regmodel_info['latest_versions'][0].run_id
# path = "/".join(regmodel_info['latest_versions'][0].source.split('/')[2:])

run_id = 'f4d1b856bb0a4abf9c8e42e6f7eb688f'
path = '/artifacts/1/019488b1b2f44793a30de8b3eeae1b72/artifacts/model'

client.download_artifacts(run_id, path, dst_path=outmodels)

{  'creation_timestamp': 1590241357691,
   'description': 'This model provides recommendation for specific users and '
                  'items based on purchase data. The data consists of user '
                  'transactions',
   'last_updated_timestamp': 1590244185655,
   'latest_versions': [  <ModelVersion: creation_timestamp=1590244182929, current_stage='Staging', description=('This model was built with Surprise library. It is a ALS based BaselineOnly '
 'algorithm'), last_updated_timestamp=1590244185655, name='Champion', run_id='f4d1b856bb0a4abf9c8e42e6f7eb688f', source='s3://mlflow/artifacts/1/f4d1b856bb0a4abf9c8e42e6f7eb688f/artifacts/model', status='READY', status_message='', user_id='', version='2'>],
   'name': 'Champion'}


MlflowException: The path of the listed S3 object does not begin with the specified artifact path. Artifact path: artifacts/1/f4d1b856bb0a4abf9c8e42e6f7eb688f/artifacts. Object path: /artifacts/1/019488b1b2f44793a30de8b3eeae1b72/artifacts/model/modeloyx7rlbx.pkl.

### Load and predict

In [ ]:
predictions, algo = dump.load(outmodels)

### Single value

In [ ]:
test_sample = data_prep_4.head()
test_sample.head()

In [ ]:
uid = str(1007)  # raw user id (as in the ratings file). They are **strings**!
iid = str(0)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo_best.predict(uid, iid, r_ui=13, verbose=True)

### Score the entire dataset

In [ ]:
data_prep_5['predictions'] = data_prep_5.apply(lambda row:algo_best.predict(row['userID'], 
                     row['itemID'], row['rating']), axis = 1)
data_prep_5.head()
# predictions=list(data_prep_5['predictions'])

### Top 10 Recommended Items


In [ ]:
def get_top_n(predictions, n=10):
    
    '''Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

#     # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_top_n_ui(top, ui):
    try:
        return {k:v for k,v in top.items() if ui==k}
    except ValueError: # user was not part of the trainset
        return 0

In [ ]:
print(get_top_n_ui(get_top_n(predictions, n=10), '20400'))